In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data"))
import zipfile
import sys
import time

from keras import backend, models, layers, initializers, regularizers, constraints, optimizers
from keras import callbacks as kc
from keras import optimizers as ko

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import log_loss
import time

['sample_submission_stage_1.csv', 'test_stage_1.tsv', 'trees']


Using TensorFlow backend.


# Load the OOFs and predictions

In [2]:
oof_filenames = os.listdir('oof')

In [3]:
[v for v in oof_filenames if 'bert-base-cased' not in v]

['bnli-mh-bert-base-uncased-seq512-8.csv',
 'bnli-mh-bert-large-cased-seq300-18.csv',
 'bnli-mh-bert-large-uncased-seq300-19.csv',
 'HBnli-mh-bert-base-uncased-seq512-8.csv',
 'HBnli-mh-bert-large-cased-seq300-18.csv',
 'HBnli-mh-bert-large-uncased-seq300-19.csv',
 'Hnaive-bert-base-uncased-seq512-8.csv',
 'Hnaive-bert-large-cased-seq300-18.csv',
 'Hnaive-bert-large-uncased-seq300-19.csv',
 'Hnli-mh-bert-base-uncased-seq512-8.csv',
 'Hnli-mh-bert-large-cased-seq300-18.csv',
 'Hnli-mh-bert-large-uncased-seq300-19.csv',
 'naive-bert-base-uncased-seq512-8.csv',
 'naive-bert-large-cased-seq300-18.csv',
 'naive-bert-large-uncased-seq300-19.csv',
 'nli-mh-bert-base-uncased-seq512-8.csv',
 'nli-mh-bert-large-cased-seq300-18.csv',
 'nli-mh-bert-large-uncased-seq300-19.csv']

In [4]:
pred_filenames = os.listdir('outputs')

In [5]:
pred_filenames

['bnli-mh-bert-base-cased-seq512-8.csv',
 'bnli-mh-bert-base-uncased-seq512-8.csv',
 'bnli-mh-bert-large-cased-seq300-18.csv',
 'bnli-mh-bert-large-uncased-seq300-19.csv',
 'HBnli-mh-bert-base-cased-seq512-8.csv',
 'HBnli-mh-bert-base-uncased-seq512-8.csv',
 'HBnli-mh-bert-large-cased-seq300-18.csv',
 'HBnli-mh-bert-large-uncased-seq300-19.csv',
 'Hnaive-bert-base-cased-seq512-8.csv',
 'Hnaive-bert-base-uncased-seq512-8.csv',
 'Hnaive-bert-large-cased-seq300-18.csv',
 'Hnaive-bert-large-uncased-seq300-19.csv',
 'Hnli-mh-bert-base-cased-seq512-8.csv',
 'Hnli-mh-bert-base-uncased-seq512-8.csv',
 'Hnli-mh-bert-large-cased-seq300-18.csv',
 'Hnli-mh-bert-large-uncased-seq300-19.csv',
 'naive-bert-base-cased-seq512-8.csv',
 'naive-bert-base-uncased-seq512-8.csv',
 'naive-bert-large-cased-seq300-18.csv',
 'naive-bert-large-uncased-seq300-19.csv',
 'nli-mh-bert-base-cased-seq512-8.csv',
 'nli-mh-bert-base-uncased-seq512-8.csv',
 'nli-mh-bert-large-cased-seq300-18.csv',
 'nli-mh-bert-large-unca

In [6]:
def get_tag(oof_name):
    if oof_name.startswith('naive'):
        return '-'.join(oof_name.split('-')[1:4])
    elif oof_name.startswith('nli'):
        return '-'.join(oof_name.split('-')[2:5])
    else:
        pritn("[WTF?]", oof_name)

In [7]:
oofs = [pd.read_csv(os.path.join('oof', v)).values for v in oof_filenames]
preds = [pd.read_csv(os.path.join('outputs', v)).values[:, 1:] for v in pred_filenames]

dev_filename = "contextual_embeddings_gap_development.json"
val_filename = "contextual_embeddings_gap_validation.json"
test_filename = "contextual_embeddings_gap_test.json"

In [8]:
remove_test = []
remove_validation = []
remove_development = [209, 1506, 1988]

In [9]:
def get_labels():
    dev = pd.read_csv('gap-development.tsv', sep = '\t')
    test =  pd.read_csv('gap-test.tsv', sep = '\t')
    validation =  pd.read_csv('gap-validation.tsv', sep = '\t')
    
    def _get_labels(df):
        labels = df[['A-coref', 'B-coref']].values
        ys = []
        for label in labels:
            y = np.zeros(3)
            if label[0]:
                y[0] = 1
            elif label[1]:
                y[1] = 1
            else:
                y[2] = 1
            ys.append(y)
        return np.array(ys)
        
    dev_labels, test_labels, valid_labels = _get_labels(dev), _get_labels(test), _get_labels(validation)  
    #dev_labels = np.delete(dev_labels, remove_development, 0) Should be change in stage 2

    test_labels = np.delete(test_labels, remove_test, 0)
    valid_labels = np.delete(valid_labels, remove_validation, 0)
    print("Shape of test, valid, dev samples", test_labels.shape, valid_labels.shape, dev_labels.shape)
    return test_labels, valid_labels, dev_labels

In [10]:
test_labels, valid_labels, dev_labels = get_labels()

Shape of test, valid, dev samples (2000, 3) (454, 3) (2000, 3)


In [11]:
test_labels

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [12]:
for oof in oofs:
    print(oof.shape)

(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)
(2454, 3)


In [13]:
for pred in preds:
    print(pred.shape)

(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)
(2000, 3)


In [14]:
train_x = np.concatenate(oofs, axis=1)
test_x = np.concatenate(preds, axis=1)
train_y = np.concatenate([test_labels, valid_labels], axis=0)
test_y = dev_labels

print("train_x shape:", train_x.shape, "train_y shape:", train_y.shape)
print("train_x shape:", test_x.shape, "train_y shape:", test_y.shape)

train_x shape: (2454, 72) train_y shape: (2454, 3)
train_x shape: (2000, 72) train_y shape: (2000, 3)


In [15]:
import keras
from keras.models import Model
from keras.layers import *
from sklearn.model_selection import KFold
from keras.optimizers import *

In [16]:
import math 
import keras.backend as K
def gelu(x):
    return 0.5 * x * (1 + K.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * K.pow(x, 3))))
class Gelu(Layer):
    def __init__(self, accurate: bool = False, **kwargs):
        super().__init__(**kwargs)
        self.accurate = accurate

    def call(self, inputs, **kwargs):
        return K.sigmoid(inputs)
        if not self.accurate:
            return gelu(inputs)
        if K.backend() == 'tensorflow':
            erf = K.tf.erf
        else:
            erf = K.T.erf
        return inputs * 0.5 * (1.0 + erf(inputs / math.sqrt(2.0)))

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = {
            'accurate': self.accurate,
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [17]:
def build_mlp_model(feature_nums):
    features_inputs = Input(shape=feature_nums, name='mata-features', dtype="float32")
    features = features_inputs
    
    depth = 5
    for i in range(depth):
        new_features = Dense(24, activation='relu')(features)
        new_features = Dropout(0.5)(new_features)
        features = Concatenate()([features, new_features])

    out_ = Dense(3, activation='softmax')(features)
    
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy',])
    model.summary()
    return model

def build_logistic_regression(feature_nums):
    features_inputs = Input(shape=feature_nums, name='mata-features', dtype="float32")
    out_ = Dense(3, activation='softmax')(features_inputs)
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy',])
    model.summary()
    return model

In [18]:
from keras.callbacks import ReduceLROnPlateau

In [19]:
prediction = np.zeros((len(test_y), 3)) # testing predictions
# Training and cross-validation
folds = KFold(n_splits=5, shuffle=True, random_state=3)
scores = []
oof = np.zeros_like(train_y)

for fold_n, (train_index, valid_index) in enumerate(folds.split(train_x)):
    X_tr, X_val = train_x[train_index], train_x[valid_index]
    Y_tr, Y_val = train_y[train_index], train_y[valid_index]
    
    classif_model = build_mlp_model([train_x.shape[-1]])
    classif_model.compile(optimizer=Adam(lr=1e-3), loss="categorical_crossentropy")

    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=5e-5),
                kc.ModelCheckpoint('stage_1_finals/nn_ensemble_checkpoint_f{}.pt'.format(fold_n), monitor='val_loss', verbose=0, save_best_only=True, mode='min')]
    # train the model
    classif_model.fit(x=X_tr, y=Y_tr, epochs=200, batch_size=32, 
                      callbacks=callbacks, validation_data=(X_val, Y_val), verbose=1)   
    
    pred_valid = classif_model.predict(x=X_val, verbose=0)
    oof[valid_index] = pred_valid
    pred = classif_model.predict(x=test_x, verbose=0)

    # oof[valid_index] = pred_valid.reshape(-1,)
    scores.append(log_loss(Y_val, pred_valid))
    prediction += pred
    
prediction /= 5   
print("-" * 30)
print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
print(scores)
#print("Test score:", log_loss(test_y, prediction)) # this line should be removed in the stage 2
print("-" * 30)    

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 72)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 24)           1752        mata-features[0][0]              
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 24)           0           dense_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 96)           0           mata-features[0][0]              
                                                                 dropout_1[0][0]                  
__________

1963/1963 [==============================] - 0s 161us/step - loss: 0.2800 - val_loss: 0.2868
Epoch 38/200
1963/1963 [==============================] - 0s 159us/step - loss: 0.2848 - val_loss: 0.2871
Epoch 39/200
1963/1963 [==============================] - 0s 160us/step - loss: 0.2759 - val_loss: 0.2871
Epoch 40/200
1963/1963 [==============================] - 0s 160us/step - loss: 0.2804 - val_loss: 0.2873
Epoch 41/200
1963/1963 [==============================] - 0s 165us/step - loss: 0.2819 - val_loss: 0.2872
Epoch 42/200
1963/1963 [==============================] - 0s 159us/step - loss: 0.2761 - val_loss: 0.2872
Epoch 43/200
1963/1963 [==============================] - 0s 162us/step - loss: 0.2748 - val_loss: 0.2871
Epoch 44/200
1963/1963 [==============================] - 0s 165us/step - loss: 0.2808 - val_loss: 0.2873
Epoch 45/200
1963/1963 [==============================] - 0s 166us/step - loss: 0.2772 - val_loss: 0.2870
Epoch 46/200
1963/1963 [==============================] - 0

1963/1963 [==============================] - 0s 167us/step - loss: 0.3021 - val_loss: 0.2778
Epoch 15/200
1963/1963 [==============================] - 0s 159us/step - loss: 0.2980 - val_loss: 0.2787
Epoch 16/200
1963/1963 [==============================] - 0s 159us/step - loss: 0.3054 - val_loss: 0.2794
Epoch 17/200
1963/1963 [==============================] - 0s 164us/step - loss: 0.2993 - val_loss: 0.2754
Epoch 18/200
1963/1963 [==============================] - 0s 164us/step - loss: 0.3010 - val_loss: 0.2802
Epoch 19/200
1963/1963 [==============================] - 0s 162us/step - loss: 0.2927 - val_loss: 0.2732
Epoch 20/200
1963/1963 [==============================] - 0s 163us/step - loss: 0.2934 - val_loss: 0.2815
Epoch 21/200
1963/1963 [==============================] - 0s 166us/step - loss: 0.2981 - val_loss: 0.2753
Epoch 22/200
1963/1963 [==============================] - 0s 159us/step - loss: 0.2949 - val_loss: 0.2694
Epoch 23/200
1963/1963 [==============================] - 0

Train on 1963 samples, validate on 491 samples
Epoch 1/200
1963/1963 [==============================] - 1s 387us/step - loss: 0.4346 - val_loss: 0.3455
Epoch 2/200
1963/1963 [==============================] - 0s 158us/step - loss: 0.3407 - val_loss: 0.3390
Epoch 3/200
1963/1963 [==============================] - 0s 157us/step - loss: 0.3218 - val_loss: 0.3344
Epoch 4/200
1963/1963 [==============================] - 0s 156us/step - loss: 0.3193 - val_loss: 0.3308
Epoch 5/200
1963/1963 [==============================] - 0s 157us/step - loss: 0.3210 - val_loss: 0.3297
Epoch 6/200
1963/1963 [==============================] - 0s 157us/step - loss: 0.3016 - val_loss: 0.3265
Epoch 7/200
1963/1963 [==============================] - 0s 158us/step - loss: 0.3089 - val_loss: 0.3246
Epoch 8/200
1963/1963 [==============================] - 0s 159us/step - loss: 0.3016 - val_loss: 0.3244
Epoch 9/200
1963/1963 [==============================] - 0s 157us/step - loss: 0.2939 - val_loss: 0.3306
Epoch 10

Epoch 78/200
1963/1963 [==============================] - 0s 162us/step - loss: 0.2602 - val_loss: 0.3150
Epoch 79/200
1963/1963 [==============================] - 0s 161us/step - loss: 0.2583 - val_loss: 0.3150
Epoch 80/200
1963/1963 [==============================] - 0s 161us/step - loss: 0.2588 - val_loss: 0.3148
Epoch 81/200
1963/1963 [==============================] - 0s 163us/step - loss: 0.2585 - val_loss: 0.3150
Epoch 82/200
1963/1963 [==============================] - 0s 162us/step - loss: 0.2564 - val_loss: 0.3150
Epoch 83/200
1963/1963 [==============================] - 0s 159us/step - loss: 0.2597 - val_loss: 0.3151
Epoch 84/200
1963/1963 [==============================] - 0s 160us/step - loss: 0.2575 - val_loss: 0.3155
Epoch 85/200
1963/1963 [==============================] - 0s 163us/step - loss: 0.2579 - val_loss: 0.3153
Epoch 86/200
1963/1963 [==============================] - 0s 161us/step - loss: 0.2575 - val_loss: 0.3153
Epoch 87/200
1963/1963 [======================

1963/1963 [==============================] - 0s 151us/step - loss: 0.2808 - val_loss: 0.3334
Epoch 19/200
1963/1963 [==============================] - 0s 151us/step - loss: 0.2838 - val_loss: 0.3328
Epoch 20/200
1963/1963 [==============================] - 0s 152us/step - loss: 0.2753 - val_loss: 0.3290
Epoch 21/200
1963/1963 [==============================] - 0s 152us/step - loss: 0.2777 - val_loss: 0.3299
Epoch 22/200
1963/1963 [==============================] - 0s 151us/step - loss: 0.2761 - val_loss: 0.3326
Epoch 23/200
1963/1963 [==============================] - 0s 151us/step - loss: 0.2677 - val_loss: 0.3292
Epoch 24/200
1963/1963 [==============================] - 0s 152us/step - loss: 0.2774 - val_loss: 0.3322
Epoch 25/200
1963/1963 [==============================] - 0s 151us/step - loss: 0.2718 - val_loss: 0.3295
Epoch 26/200
1963/1963 [==============================] - 0s 150us/step - loss: 0.2681 - val_loss: 0.3293
Epoch 27/200
1963/1963 [==============================] - 0

Train on 1964 samples, validate on 490 samples
Epoch 1/200
1964/1964 [==============================] - 1s 469us/step - loss: 0.5038 - val_loss: 0.3199
Epoch 2/200
1964/1964 [==============================] - 0s 169us/step - loss: 0.3506 - val_loss: 0.2975
Epoch 3/200
1964/1964 [==============================] - 0s 167us/step - loss: 0.3481 - val_loss: 0.2896
Epoch 4/200
1964/1964 [==============================] - 0s 166us/step - loss: 0.3339 - val_loss: 0.2956
Epoch 5/200
1964/1964 [==============================] - 0s 166us/step - loss: 0.3225 - val_loss: 0.2929
Epoch 6/200
1964/1964 [==============================] - 0s 168us/step - loss: 0.3221 - val_loss: 0.2914
Epoch 7/200
1964/1964 [==============================] - 0s 169us/step - loss: 0.3184 - val_loss: 0.2902
Epoch 8/200
1964/1964 [==============================] - 0s 164us/step - loss: 0.3147 - val_loss: 0.2918
Epoch 9/200
1964/1964 [==============================] - 0s 163us/step - loss: 0.3125 - val_loss: 0.2898
Epoch 10

In [22]:
#print("Test score:", log_loss(test_y, prediction/5)) # this line should be removed in the stage 2

Test score: 0.2992591341007771


In [23]:
# make a submission
submission = pd.read_csv("data/sample_submission_stage_1.csv", index_col = "ID")
submission["A"] = prediction[:,0]
submission["B"] = prediction[:,1]
submission["NEITHER"] = prediction[:,2]
submission.to_csv("stage_1_finals/24_nn_ensembles.csv")